<a href="https://colab.research.google.com/github/jessfeld/netpyne_course_2021/blob/main/Copy_of_tut_netpyne_artif_stim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install neuron
!pip install netpyne
import matplotlib

In [ ]:
%matplotlib inline

In [ ]:
!git clone https://github.com/suny-downstate-medical-center/netpyne-course-2021.git
#!rm x86_64
!nrnivmodl netpyne-course-2021
#from neuron import load_mechanisms
#try:
#  load_mechanisms('.')
#except:
#  print('already loaded')

fatal: destination path 'netpyne-course-2021' already exists and is not an empty directory.
/content
Mod files: "netpyne-course-2021/vecevent.mod"

COBJS=''
 -> Compiling mod_func.c
x86_64-linux-gnu-gcc -O2   -I.   -I/usr/local/lib/python3.7/dist-packages/neuron/.data/include  -I/nrnwheel/openmpi/include -fPIC -c mod_func.c -o mod_func.o
 => LINKING shared library ./libnrnmech.so
x86_64-linux-gnu-g++ -O2 -DVERSION_INFO='8.0.0' -std=c++11 -shared -fPIC  -I /usr/local/lib/python3.7/dist-packages/neuron/.data/include -o ./libnrnmech.so -Wl,-soname,libnrnmech.so \
  ./mod_func.o ./vecevent.o  -L/usr/local/lib/python3.7/dist-packages/neuron/.data/lib -lnrniv -Wl,-rpath,/usr/local/lib/python3.7/dist-packages/neuron/.data/lib   
rm -f ./.libs/libnrnmech.so ; mkdir -p ./.libs ; cp ./libnrnmech.so ./.libs/libnrnmech.so
Successfully created x86_64/special


In [ ]:
"""
tut_artif.py

Tutorial on artificial cells (no sections)
"""

from netpyne import specs, sim
from netpyne.specs import Dict

netParams = specs.NetParams()  # object of class NetParams to store the network parameters
simConfig = specs.SimConfig()  # dictionary to store sets of simulation configurations


###############################################################################
# NETWORK PARAMETERS
###############################################################################
# Cell parameters
## PYR cell properties
cellParams = Dict()
cellParams.secs.soma.geom = {'diam': 18.8, 'L': 18.8, 'Ra': 123.0}
cellParams.secs.soma.mechs.hh = {'gnabar': 0.12, 'gkbar': 0.036, 'gl': 0.003, 'el': -70}
netParams.cellParams['PYR'] = cellParams

## IntFire2 artificial cell
netParams.cellParams['artif_IntFire2'] = {
    'cellModel': 'IntFire2', 
    'ib': 0.0}  

## IntFire4 artificial cell
netParams.cellParams['artif_IntFire4'] = {
    'cellModel': 'IntFire4', 
    'taue': 1.0}

## NetStim artificial spike generator
netParams.cellParams['artif_NetStim'] = {
    'cellModel': 'NetStim'}
    
## VecStim artificial spike generator
netParams.cellParams['artif_VecStim'] = {
    'cellModel': 'VecStim'}  # pop of Vecstims with 2 pulses


# Population parameters
netParams.popParams['PYR1'] = {
    'cellType': 'PYR', 
    'numCells': 100} # pop of HH cells

netParams.popParams['pop_IntFire2'] = {
    'cellType': 'artif_IntFire2', 
    'numCells': 100}  # pop of IntFire2

netParams.popParams['pop_IntFire4'] = {
    'cellType': 'artif_IntFire4', 
    'numCells': 100}  # pop of IntFire4

netParams.popParams['pop_NetStim'] = {
    'cellType': 'artif_NetStim', 
    'numCells': 100,
    'rate': 10, 
    'noise': 0.8, 
    'start': 1, 
    'seed': 2}  # pop of NEtSims

netParams.popParams['pop_VecStim'] = {
    'cellType': 'artif_VecStim', 
    'numCells': 100, 
    'rate': 5, 
    'noise': 0.5, 
    'start': 50,
    'pulses': [{'start': 200, 'end': 300, 'rate': 60, 'noise':0.2}, 
               {'start': 500, 'end': 800, 'rate': 30, 'noise': 0.5}]}  # pop of Vecstims with 2 pulses


# Synaptic mechanism parameters
netParams.synMechParams['AMPA'] = {'mod': 'Exp2Syn', 'tau1': 0.1, 'tau2': 1.0, 'e': 0}


# Connections
netParams.connParams['NetStim->PYR1'] = {
    'preConds': {'pop': 'pop_NetStim'}, 
    'postConds': {'pop': 'PYR1'},
    'convergence': 3,
    'weight': 0.002,
    'synMech': 'AMPA',
    'delay': 'uniform(1,5)'}


netParams.connParams['VecStim->PYR1'] = {
    'preConds': {'pop': 'pop_VecStim'}, 
    'postConds': {'pop': 'PYR1'},
    'probability': 0.4,
    'weight': 0.005,
    'synMech': 'AMPA',
    'delay': 'uniform(1,5)'}

netParams.connParams['PYR1->IntFire2'] = {
    'preConds': {'pop': 'PYR1'}, 
    'postConds': {'pop': 'pop_IntFire2'},
    'probability': 0.2,
    'weight': 0.1,
    'synMech': 'AMPA',
    'delay': 'uniform(1,5)'}


netParams.connParams['IntFire2->IntFire4'] = {
    'preConds': {'pop': 'pop_IntFire2'}, 
    'postConds': {'pop': 'pop_IntFire4'},
    'probability': 0.1,
    'weight': 0.2,
    'delay': 'uniform(1,5)'}


###############################################################################
# SIMULATION PARAMETERS
###############################################################################

# Simulation parameters
simConfig.duration = 1*1e3 # Duration of the simulation, in ms
simConfig.dt = 0.1 # Internal integration timestep to use
simConfig.createNEURONObj = 1  # create HOC objects when instantiating network
simConfig.createPyStruct = 1  # create Python structure (simulator-independent) when instantiating network
simConfig.verbose = False  # show detailed messages

# Recording
simConfig.recordTraces = {'Vsoma':{'sec':'soma','loc':0.5,'var':'v'}}

# # Analysis and plotting
simConfig.analysis['plotRaster'] = {'orderInverse': True, 'saveFig': True}


###############################################################################
# RUN SIM
###############################################################################

sim.createSimulateAnalyze()


Streaming output truncated to the last 5000 lines.
  Created connection preGid=454, postGid=69, sec=soma, loc=0.5, synMech=AMPA, weight=0.005, delay=4.16
  Created connection preGid=456, postGid=69, sec=soma, loc=0.5, synMech=AMPA, weight=0.005, delay=4.67
  Created connection preGid=459, postGid=69, sec=soma, loc=0.5, synMech=AMPA, weight=0.005, delay=3.85
  Created connection preGid=460, postGid=69, sec=soma, loc=0.5, synMech=AMPA, weight=0.005, delay=4.63
  Created connection preGid=461, postGid=69, sec=soma, loc=0.5, synMech=AMPA, weight=0.005, delay=1.26
  Created connection preGid=462, postGid=69, sec=soma, loc=0.5, synMech=AMPA, weight=0.005, delay=1.56
  Created connection preGid=469, postGid=69, sec=soma, loc=0.5, synMech=AMPA, weight=0.005, delay=3.74
  Created connection preGid=471, postGid=69, sec=soma, loc=0.5, synMech=AMPA, weight=0.005, delay=4.80
  Created connection preGid=473, postGid=69, sec=soma, loc=0.5, synMech=AMPA, weight=0.005, delay=1.56
  Created connection p

1) Make a network